This project implements transfer learning using the VGG16 pre-trained model on a subset of the CIFAR-10 dataset, and adapting the model for image classification.

# Step 1: Import necessary libraries and load dataset

Imports the required libraries for TensorFlow and Keras.
Loads a small subset (1%) of the CIFAR-10 dataset using TensorFlow Datasets.
Splits the data into training (1%) and validation (29%) sets.

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.applications import VGG16
import tensorflow_datasets as tfds

# Load CIFAR-10 dataset from TensorFlow Datasets
(train_ds, validation_ds), info = tfds.load(
    'cifar10',
    split=['train[:1%]', 'train[1%:2%]'],
    with_info=True,
    as_supervised=True
)

# Step 2: Preprocess the Data

Define a function preprocess_image to resize and preprocess images.
The function is designed to be applied to each image-label pair in the dataset.

In [ ]:
# Preprocess the data
def preprocess_image(image, label):
    image = tf.image.resize(image, (224, 224))
    image = tf.keras.applications.vgg16.preprocess_input(image)
    return image, label

# Step 3: Apply preprocessing to datasets

* Apply the defined preprocessing function to both training and validation
datasets
* Batch the data into groups of 32 for more efficient processing
* Use prefetching for optimizing data loading

In [ ]:
train_ds = train_ds.map(preprocess_image).batch(32).prefetch(tf.data.AUTOTUNE)
validation_ds = validation_ds.map(preprocess_image).batch(32).prefetch(tf.data.AUTOTUNE)

# Step 4: Instantiate model

* Instantiate the VGG16 model with pre-trained weights on ImageNet
* include_top=False excludes the fully-connected layers at the top of the model.

In [ ]:
base_model = VGG16(input_shape=(224, 224, 3),
                   include_top=False,
                   weights='imagenet')

58889256/58889256 [==============================] - 0s 0us/step


#Step 5: Freeze the layers of the pre-trained model

Freezes all layers of the pre-trained VGG16 model to retain their learned features during transfer learning.

In [ ]:
for layer in base_model.layers:
    layer.trainable = False

# Step 6: Create a custom model

* Create a new model by adding custom classification layers on top of the frozen VGG16 model
* The architecture includes a flattening layer, a dense layer with ReLU activation, a dropout layer, and a final dense layer with softmax activation for 10 classes

In [ ]:
x = layers.Flatten()(base_model.output)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(10, activation='softmax')(x)  # CIFAR-10 has 10 classes

model = Model(base_model.input, x)

# Step 7: Compile the model

Compile the model with the Adam optimizer, sparse categorical crossentropy loss, and accuracy as the metric.

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Step 8: Train the model

* Train the model on the preprocessed training dataset for 3 epochs
* Use the validation dataset for monitoring the model's performance during training

In [ ]:
history = model.fit(train_ds,
                    epochs=3,
                    validation_data=validation_ds)

Epoch 1/3
14/16 [=========================>....] - ETA: 39s - loss: 9.2982 - accuracy: 0.2634

# Step 9: Save the model

Save the trained model to a file named 'transfer_learning_model_cifar10.h5' for future use.

In [ ]:
model.save('transfer_learning_model_cifar10.h5')